In [1]:
# h5ファイルをnumpy.array形式で読み出して、c++用のコードに変換する
# 2018/8/1 haruta

In [7]:
# 定義
filename = 'model_0811_haruta2.h5' # h5ファイル名

# それぞれの保管パス
w1 = 'model_weights/dense_1/dense_1/kernel:0'
w2 = 'model_weights/dense_2/dense_2/kernel:0'
w3 = 'model_weights/dense_3/dense_3/kernel:0'

b1 = 'model_weights/dense_1/dense_1/bias:0'
b2 = 'model_weights/dense_2/dense_2/bias:0'
b3 = 'model_weights/dense_3/dense_3/bias:0'

# 保存先のフォルダ名
savefolder = 'weights_txt/'

import h5py    
import numpy as np
np.set_printoptions(threshold=np.inf)
f1 = h5py.File(filename,'r+')  

In [3]:
# メモ用


In [5]:
# アドレス
# List all groups
a_group_key1 = list(f1.keys())[0] # ここで0なら通常model_weightsが出力される
print("Keys1: %s" % a_group_key1)

# Get the data
a_group_key2 = list(f1[a_group_key1]) # この中からdense_XX, dense_XX+1, dense_XX+2が該当する
print("Keys2: %s" % a_group_key2)

# Get the data
# この中からdense_XX, dense_XX+1, dense_XX+2が該当する
a_group_key3 = list(f1[a_group_key1 + '/dense_1']) 
print("Keys3: %s" % a_group_key3)

key3 = '/dense_1' # dense_XX ~ dense_XX+2 のどれかを入れる

address = a_group_key1 + '/dense_1' + '/dense_1'+ '/kernel:0' # kernel:0もしくはbias:0
print("address: %s" % address)
value = list(f1[address]) 
#len(value)

Keys1: model_weights
Keys2: ['activation_1', 'activation_2', 'activation_3', 'dense_1', 'dense_2', 'dense_3', 'dropout_1', 'dropout_2']
Keys3: ['dense_1']
address: model_weights/dense_1/dense_1/kernel:0


In [8]:
# それぞれの変数をnp.array形式で抽出
w1_values = np.array(f1[w1]).astype("float32")
w2_values = np.array(f1[w2]).astype("float32")
w3_values = np.array(f1[w3]).astype("float32")

b1_values = np.array(f1[b1]).astype("float32")
b2_values = np.array(f1[b2]).astype("float32")
b3_values = np.array(f1[b3]).astype("float32")

print('OK')

OK


In [9]:
print(w1_values.shape) #これで(516,380)だったら転置する
print(w2_values.shape)
print(w3_values.shape)
print(b1_values.shape)
print(b2_values.shape)
print(b3_values.shape)

(516, 380)
(380, 257)
(257, 129)
(380,)
(257,)
(129,)


In [10]:
w1_values_tp = np.transpose(w1_values)
w1_values_tp.shape #これで(380,516)になれば良い
w2_values_tp = np.transpose(w2_values)
w3_values_tp = np.transpose(w3_values)

In [11]:
def savetxt(savefolder, txtname, data,varname):
    '''「savefolder」フォルダに、ファイル名「txtname」で、変数dataを{{1行目},{2行目},...}の形式で書き込む。
    （この形式だとC++にコピペした時に同じ変数配列として扱われる。）varnameはC++で使用したい変数名なので、
    変数名[行数][列数]の形式でかく'''
    savepath = savefolder + txtname
    with open(savepath, "w") as f:
        print(varname +'='+np.array2string(data, separator=', ', formatter={'float_kind': lambda x: '{: .10f}'.format(x)}).replace('\n','').replace('[','{').replace(']','}')+';',file=f,end='')
    return 1


In [12]:
#テキスト保存
savetxt(savefolder, 'w1.txt', w1_values_tp,'W1_before[380][516]')
savetxt(savefolder, 'w2.txt', w2_values_tp,'W2_before[257][380]')
savetxt(savefolder, 'w3.txt', w3_values_tp,'W3_before[129][257]')

savetxt(savefolder, 'b1.txt', b1_values,'b1_before[380]')
savetxt(savefolder, 'b2.txt', b2_values,'b2_before[257]')
savetxt(savefolder, 'b3.txt', b3_values,'b3_before[129]')
print('OK')

OK


In [116]:
print(w1_values_tp[0,0:5])
print(w2_values_tp[0,0:5])
print(w3_values_tp[0,0:5])
print('')
print(b1_values[0:5])
print(b2_values[0:5])
print(b3_values[0:5])

[ 0.36844915 -0.33617136 -0.4528113  -0.467853    0.19404523]
[ 0.13534664 -0.14093459 -0.40718347  0.22906736  0.20009875]
[ 0.06111534 -0.02756482  0.06526674  0.22819003  0.08436346]

[-0.02649925 -0.01293557 -0.00613426  0.00122955  0.00497521]
[ 0.00260683 -0.04817585 -0.00885866  0.02250442 -0.05931728]
[-3.5303118  -1.651823   -0.27213216  1.364964    2.2860272 ]


In [58]:
# 
# --------------------------------------------------------------------------------------
# 以下、確認用(実行しなくても動作には影響ない)　コード間違っている可能性あり

In [79]:
data_code = ...
'W1_before[516][380] = ' + str(b1_values).replace('\n','') + ";" + '\n' +...
'W1_before[380][257] = ' + str(b1_values).replace('\n','')+ ";" + '\n' +...
'W1_before[257][129] = ' + str(b1_values).replace('\n','') + ";" + '\n'

SyntaxError: unexpected character after line continuation character (<ipython-input-79-e102730b9b4b>, line 2)

In [9]:
# この結果が'model_weights'であることを確認する。違ったらインデックス変える
a_group_key = list(f1.keys())[group_list]
print(a_group_key)

NameError: name 'group_list' is not defined

In [3]:
# インデックス 初期値で問題なければ変えない
group_list=0
var_list = 0

In [36]:
var = np.array(list(list(f1.values()))[0])
print(var)

['activation_7' 'activation_8' 'activation_9' 'batch_normalization_5'
 'batch_normalization_6' 'dense_7' 'dense_8' 'dense_9' 'dropout_3']


In [6]:
# 確認用に表示
np.array(list(list(f1.values()))[group_list])

array(['activation_7', 'activation_8', 'activation_9',
       'batch_normalization_5', 'batch_normalization_6', 'dense_7',
       'dense_8', 'dense_9', 'dropout_3'],
      dtype='<U21')

In [5]:
list(f1.keys())

['model_weights', 'optimizer_weights']